In [1]:
import os
import cv2
import numpy as np

from utils.dataloader import DataLoader
from utils.vis import MatplotlibVisualizer
from descriptors.color import ColorDescriptor, ColorLayoutDescriptor, ColorCooccurrenceMatrixDescriptor
from descriptors.sift import SIFTDescriptor

In [2]:
## Classes
CLASSES = ['nevus', 'others']

## Work folfer
work_folder = os.getcwd()
data_folder = os.path.join(work_folder, '..', 'Data/Challenge1')

## Visualizer
matplotlib_visualizer = MatplotlibVisualizer()

## Featrure Extraction

### Color Descriptor

In [ ]:
modes = ['train', 'val']
max_samples = 200

for mode in modes:
    ## Make a new folder for the features
    os.makedirs(os.path.join('feautres', mode, 'color'), exist_ok=True)

    ## Data loader
    dataloader = DataLoader(data_folder, mode, shuffle=True, ignore_folders=['black_background', '.DS_Store'])

    ## Descriptors
    color_descriptor = ColorLayoutDescriptor((8,8))
    color_cooccurrence_matrix_descriptor = ColorCooccurrenceMatrixDescriptor(distances=[1], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4], levels=8)

    ## Extract features
    features = []
    labels = []
    for i, (img, label, path) in enumerate(dataloader):
        if i >= max_samples and mode == 'train':
            break
        
        ## combine all descriptors
        color_features = color_descriptor.extract(img)
        color_cooccurrence_matrix_features = color_cooccurrence_matrix_descriptor.extract(img)
        features.append(np.concatenate([color_features, color_cooccurrence_matrix_features]))
        
        ## add label
        labels.append(label)

    ## Save features to disk
    features = np.array(features)
    labels = np.array(labels)
    features_with_labels = np.concatenate([features, labels.reshape(-1, 1)], axis=1)
    np.save(os.path.join('feautres', mode, 'color', 'features.npy'), features_with_labels)

## Training

In [14]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [51]:
mode = 'train'
svm = SVC(kernel='linear', C=1.0, random_state=42)

In [52]:
features_with_labels = np.load(os.path.join('feautres', mode, 'color', 'features.npy'))
features = features_with_labels[:, :-1]
labels = features_with_labels[:, -1]

X_train, X_val, y_train, y_val = train_test_split(features, labels, test_size=0.2, random_state=42)

svm.fit(X_train, y_train)

SVC(kernel='linear', random_state=42)

## Validation

In [53]:
y_pred = svm.predict(X_val)
print(classification_report(y_val, y_pred, target_names=CLASSES))

              precision    recall  f1-score   support

       nevus       0.94      0.61      0.74        28
      others       0.50      0.92      0.65        12

    accuracy                           0.70        40
   macro avg       0.72      0.76      0.69        40
weighted avg       0.81      0.70      0.71        40



## Testing

In [54]:
mode = 'val'
features_with_labels = np.load(os.path.join('feautres', mode, 'color', 'features.npy'))
features = features_with_labels[:, :-1]
labels = features_with_labels[:, -1]

y_pred = svm.predict(features)
print(classification_report(labels, y_pred, target_names=CLASSES))

              precision    recall  f1-score   support

       nevus       0.60      0.44      0.51      1052
      others       0.68      0.80      0.74      1583

    accuracy                           0.66      2635
   macro avg       0.64      0.62      0.62      2635
weighted avg       0.65      0.66      0.65      2635

